# 1. **Preparing Process**

---

In [ ]:
import random
import re
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from resnet import ResNet
from time import time
from contextlib import contextmanager
import os
from torch.cuda import max_memory_allocated

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [ ]:
@contextmanager
def print_and_log(log_filename):
    log_file = open(log_filename, 'w')
    def log(message):
        print(message)
        log_file.write(message + '\n')
        log_file.flush()
    yield log
    log_file.close()

class ApplyTransform(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return self.transform(x), y

    def __len__(self):
        return len(self.dataset)

# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

TRANSFORM_TRAIN = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    normalize,
])
TRANSFORM_TEST = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

TRAIN_SET = datasets.CIFAR10(root='./data', train=True, download=True, transform=TRANSFORM_TRAIN)
TEST_SET = datasets.CIFAR10(root='./data', train=False, download=True, transform=TRANSFORM_TEST)







Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def test(model: nn.Module, loader: DataLoader):
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    test_correct = 0
    test_total = 0
    start_time = time()
    torch.cuda.reset_max_memory_allocated(DEVICE)

    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            test_correct += (outputs.argmax(1) == targets).sum().item()
            test_total += targets.size(0)

    latency = time() - start_time
    max_memory_usage = max_memory_allocated(DEVICE) / (1024 ** 2)

    test_loss /= len(loader)
    test_acc = test_correct / test_total
    return test_loss, test_acc, latency, max_memory_usage

In [ ]:
def train(w_nbits, a_nbits, lr=0.1, weight_decay=1e-3,
          optimizer_type='sgd', epochs=200, batch_size=128):

    # import resnet20
    # model = resnet20.ResNetCIFAR(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)
    torch.cuda.reset_max_memory_allocated(DEVICE)
    model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits).to(DEVICE)

    ckpt_base_filename = (
        'resnet20_cifar'
        + ('' if w_nbits is None else f'_wq{w_nbits}')
        + ('' if a_nbits is None else f'_aq{a_nbits}')
        + '.pt'
    )
    log_filename = ckpt_base_filename.replace('.pt', '.txt')
    last_saved_filename = None

    train_loader = DataLoader(TRAIN_SET, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TEST_SET, batch_size=batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()

    if optimizer_type.lower() == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type.lower() == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type.lower() == 'adagrad':
        optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150], gamma=0.1)

    best_test_acc = 0.0
    save_threshold_epoch = min(50, epochs // 3)


    with print_and_log(log_filename) as log:
        log(f'\nQuantization: weight={w_nbits} activation={a_nbits}, Using device: {DEVICE}')


        for epoch in range(epochs):
            log(f'\nEpoch {epoch+1}')
            model.train()

            train_loss = 0
            train_correct = 0
            train_total = 0

            start_time = time()

            for inputs, targets in tqdm(train_loader, desc='Train', unit='batch', ascii=True, dynamic_ncols=True):
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == targets).sum().item()
                train_total += targets.size(0)

            train_loss /= len(train_loader)
            train_acc = train_correct / train_total

            train_latency = time() - start_time
            max_memory_usage_during_training = max_memory_allocated(DEVICE) / (1024 ** 2)

            log(f'LR: {optimizer.param_groups[0]["lr"]:.4e}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train Latency: {train_latency:.2f}s, Memory Usage: {max_memory_usage_during_training:.2f}MB')

            model.eval()
            test_loss, test_acc, test_latency, test_memory_usage = test(model, test_loader)
            log(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Latency: {test_latency:.2f}s, Test Memory Usage: {test_memory_usage:.2f}MB')

            scheduler.step()

            if epoch >= save_threshold_epoch:
                if test_acc > best_test_acc:
                    best_test_acc = test_acc
                    print('Saving best model ...')
                    if last_saved_filename is not None and last_saved_filename != ckpt_base_filename:
                        os.remove(last_saved_filename)
                    ckpt_filename = f'epoch_{epoch+1}_' + ckpt_base_filename
                    torch.save(model.state_dict(), ckpt_filename)
                    last_saved_filename = ckpt_filename

    print('Saving final epoch model ...')
    torch.save(model.state_dict(), ckpt_base_filename)
    return ckpt_base_filename

In [ ]:
from lq_layers import LQLinear, LQConv, LQActiv
import numpy as np
import matplotlib.pyplot as plt


def parse_quantization_bits_from_filename(filename):
    match_w = re.search(r'_wq(\d+)', filename)
    match_a = re.search(r'_aq(\d+)', filename)

    w_nbits = int(match_w.group(1)) if match_w else None
    a_nbits = int(match_a.group(1)) if match_a else None
    base_filename = os.path.splitext(os.path.basename(filename))[0]

    print('weight bits:', w_nbits, 'activation bits:', a_nbits, 'base filename:', base_filename)

    return w_nbits, a_nbits, base_filename

def load_model(model_path, num_layers, num_classes, w_nbits, a_nbits, device):
    model = ResNet(w_nbits=w_nbits, a_nbits=a_nbits)
    model.to(device)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    return model

def collect_activations_and_quantized_weights(model, dataloader, device):
    activations = {}
    quantized_weights = {}

    def get_quantized_weight(name, layer):
        def hook(model, input, output):
            q_weight = layer.lq.apply(layer.conv.weight, layer.basis, False)[0]
            quantized_weights[name] = q_weight.detach().cpu().numpy()
        return hook

    def get_activation(name, layer):
      def hook(model, input, output):
          activations[name] = output.detach().cpu().numpy()
      return hook

    for name, layer in model.named_modules():
        if isinstance(layer, LQConv):
            layer.register_forward_hook(get_quantized_weight(name, layer))
        if isinstance(layer, LQActiv):
            layer.register_forward_hook(get_activation(name, layer))

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            _ = model(inputs)

    return activations, quantized_weights

def plot_distributions_side_by_side(activations, quantized_weights, base_filename):
    sorted_weight_keys = sorted(quantized_weights.keys())
    sorted_activation_keys = sorted(activations.keys())

    num_layers = max(len(sorted_weight_keys), len(sorted_activation_keys))
    fig, axes = plt.subplots(num_layers, 2, figsize=(12, 6 * num_layers))

    output_directory = f"./output/{base_filename}"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for i, (weight_key, activation_key) in enumerate(zip(sorted_weight_keys, sorted_activation_keys)):
        ax_w = axes[i][0] if num_layers > 1 else axes[0]
        ax_a = axes[i][1] if num_layers > 1 else axes[1]
        ax_w.hist(quantized_weights[weight_key].ravel(), bins=50, alpha=0.75)
        ax_w.set_title(f'Quantized Weight Distribution: {weight_key}')
        ax_w.set_xlabel('Weight Values')
        ax_w.set_ylabel('Frequency')

        ax_a.hist(activations[activation_key].ravel(), bins=50, alpha=0.75)
        ax_a.set_title(f'Activation Distribution: {activation_key}')
        ax_a.set_xlabel('Activation Values')
        ax_a.set_ylabel('Frequency')

        plot_filename = f"{output_directory}/{base_filename}_{weight_key}_and_{activation_key}.png"
        plt.savefig(plot_filename)
        print(f"Saved plot as {plot_filename}")

    plt.close(fig)

def analyze_model(model_filename, device='cuda'):
    w_nbits, a_nbits, base_filename = parse_quantization_bits_from_filename(model_filename)
    num_layers = 20
    num_classes = 10

    model = load_model(model_filename, num_layers, num_classes, w_nbits, a_nbits, device)
    test_loader = DataLoader(TEST_SET, batch_size=64, shuffle=False)

    activations, weights = collect_activations_and_quantized_weights(model, test_loader, device)
    plot_distributions_side_by_side(activations, weights, base_filename)




# 2. **Base Model**

---


In [ ]:
model_name = train(w_nbits=None, a_nbits=None)

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(



Quantization: weight=None activation=None, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:25<00:00, 15.04batch/s]


LR: 1.0000e-01, Train Loss: 1.5436, Train Acc: 0.4311, Train Latency: 26.00s, Memory Usage: 291.35MB
Test Loss: 1.3381, Test Acc: 0.5248, Test Latency: 2.83s, Test Memory Usage: 61.86MB

Epoch 2


Train: 100%|##########| 391/391 [00:24<00:00, 15.66batch/s]


...

LR: 1.0000e-04, Train Loss: 0.0371, Train Acc: 0.9925, Train Latency: 25.57s, Memory Usage: 212.38MB
Test Loss: 0.2809, Test Acc: 0.9186, Test Latency: 2.86s, Test Memory Usage: 61.86MB

Epoch 199


Train: 100%|##########| 391/391 [00:25<00:00, 15.28batch/s]


LR: 1.0000e-04, Train Loss: 0.0376, Train Acc: 0.9921, Train Latency: 25.60s, Memory Usage: 212.38MB
Test Loss: 0.2813, Test Acc: 0.9201, Test Latency: 2.85s, Test Memory Usage: 66.23MB

Epoch 200


Train: 100%|##########| 391/391 [00:25<00:00, 15.44batch/s]


LR: 1.0000e-04, Train Loss: 0.0368, Train Acc: 0.9921, Train Latency: 25.32s, Memory Usage: 212.38MB
Test Loss: 0.2816, Test Acc: 0.9194, Test Latency: 2.71s, Test Memory Usage: 61.86MB
Saving final epoch model ...


In [ ]:
analyze_model(model_name)

weight bits: None activation bits: None base filename: resnet20_cifar
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer2.1.conv1_and_layer2.1.activ1.1.png
Saved plot as ./output/resnet20_cifar/resnet20_cifar_layer2.1.conv2_and_layer2.1.act

# **Weight Quantization**

---

In [ ]:
model_name_list = []
for w_nbits in [3, 2, 1]:
    temp_name = train(w_nbits=w_nbits, a_nbits=None)
    model_name_list.append(temp_name)


Quantization: weight=3 activation=None, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:41<00:00,  9.32batch/s]


LR: 1.0000e-01, Train Loss: 1.6972, Train Acc: 0.3707, Train Latency: 41.98s, Memory Usage: 214.43MB
Test Loss: 1.5643, Test Acc: 0.4673, Test Latency: 5.52s, Test Memory Usage: 62.89MB

Epoch 2


Train: 100%|##########| 391/391 [00:40<00:00,  9.54batch/s]


LR: 1.0000e-01, Train Loss: 1.2098, Train Acc: 0.5590, Train Latency: 40.97s, Memory Usage: 214.43MB
Test Loss: 2.2502, Test Acc: 0.3901, Test Latency: 5.50s, Test Memory Usage: 62.89MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.2103, Train Acc: 0.9291, Train Latency: 31.30s, Memory Usage: 214.45MB
Test Loss: 0.4048, Test Acc: 0.8671, Test Latency: 3.60s, Test Memory Usage: 62.91MB

Epoch 199


Train: 100%|##########| 391/391 [00:31<00:00, 12.55batch/s]


LR: 1.0000e-04, Train Loss: 0.2111, Train Acc: 0.9288, Train Latency: 31.16s, Memory Usage: 214.45MB
Test Loss: 0.3593, Test Acc: 0.8816, Test Latency: 3.56s, Test Memory Usage: 62.91MB

Epoch 200


Train: 100%|##########| 391/391 [00:31<00:00, 12.55batch/s]


LR: 1.0000e-04, Train Loss: 0.2132, Train Acc: 0.9281, Train Latency: 31.16s, Memory Usage: 214.45MB
Test Loss: 0.3600, Test Acc: 0.8818, Test Latency: 3.58s, Test Memory Usage: 62.91MB
Saving final epoch model ...


In [ ]:
for model_name in model_name_list:
    print('_'*50)
    analyze_model(model_name)
    print('_'*50)

__________________________________________________
weight bits: 3 activation bits: None base filename: resnet20_cifar_wq3
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3/resnet20_cifar_wq3_l

# **Weight & Activation**

---

In [11]:
model_name = train(w_nbits=1, a_nbits=2)


Quantization: weight=1 activation=2, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:44<00:00,  8.79batch/s]


LR: 1.0000e-01, Train Loss: 1.8263, Train Acc: 0.3060, Train Latency: 44.50s, Memory Usage: 330.04MB
Test Loss: 1.5990, Test Acc: 0.4078, Test Latency: 5.48s, Test Memory Usage: 113.18MB

Epoch 2


Train: 100%|##########| 391/391 [00:44<00:00,  8.85batch/s]


LR: 1.0000e-01, Train Loss: 1.5024, Train Acc: 0.4442, Train Latency: 44.21s, Memory Usage: 330.32MB
Test Loss: 1.3360, Test Acc: 0.4970, Test Latency: 5.44s, Test Memory Usage: 115.45MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.5545, Train Acc: 0.8104, Train Latency: 44.14s, Memory Usage: 327.34MB
Test Loss: 0.6074, Test Acc: 0.7885, Test Latency: 5.37s, Test Memory Usage: 126.67MB

Epoch 199


Train: 100%|##########| 391/391 [00:44<00:00,  8.86batch/s]


LR: 1.0000e-04, Train Loss: 0.5586, Train Acc: 0.8089, Train Latency: 44.14s, Memory Usage: 327.32MB
Test Loss: 0.5827, Test Acc: 0.7968, Test Latency: 5.40s, Test Memory Usage: 127.27MB

Epoch 200


Train: 100%|##########| 391/391 [00:44<00:00,  8.82batch/s]


LR: 1.0000e-04, Train Loss: 0.5632, Train Acc: 0.8082, Train Latency: 44.33s, Memory Usage: 327.31MB
Test Loss: 0.5874, Test Acc: 0.7968, Test Latency: 5.46s, Test Memory Usage: 127.50MB
Saving final epoch model ...


In [12]:
analyze_model(model_name)

weight bits: 1 activation bits: 2 base filename: resnet20_cifar_wq1_aq2
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/resnet20_cifar_wq1_aq2_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq1_aq2/re

In [13]:
model_name = train(w_nbits=2, a_nbits=2)


Quantization: weight=2 activation=2, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:47<00:00,  8.23batch/s]


LR: 1.0000e-01, Train Loss: 1.8261, Train Acc: 0.3132, Train Latency: 47.54s, Memory Usage: 329.89MB
Test Loss: 1.5551, Test Acc: 0.4147, Test Latency: 6.02s, Test Memory Usage: 114.55MB

Epoch 2


Train: 100%|##########| 391/391 [00:47<00:00,  8.28batch/s]


LR: 1.0000e-01, Train Loss: 1.4940, Train Acc: 0.4469, Train Latency: 47.21s, Memory Usage: 330.43MB
Test Loss: 1.5566, Test Acc: 0.4253, Test Latency: 6.02s, Test Memory Usage: 118.55MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.4764, Train Acc: 0.8355, Train Latency: 47.48s, Memory Usage: 327.32MB
Test Loss: 0.5142, Test Acc: 0.8237, Test Latency: 6.05s, Test Memory Usage: 132.62MB

Epoch 199


Train: 100%|##########| 391/391 [00:47<00:00,  8.23batch/s]


LR: 1.0000e-04, Train Loss: 0.4753, Train Acc: 0.8370, Train Latency: 47.49s, Memory Usage: 327.35MB
Test Loss: 0.5176, Test Acc: 0.8231, Test Latency: 6.08s, Test Memory Usage: 132.55MB

Epoch 200


Train: 100%|##########| 391/391 [00:47<00:00,  8.25batch/s]


LR: 1.0000e-04, Train Loss: 0.4743, Train Acc: 0.8372, Train Latency: 47.37s, Memory Usage: 327.34MB
Test Loss: 0.5148, Test Acc: 0.8223, Test Latency: 6.06s, Test Memory Usage: 132.55MB
Saving final epoch model ...


In [14]:
analyze_model(model_name)

weight bits: 2 activation bits: 2 base filename: resnet20_cifar_wq2_aq2
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/resnet20_cifar_wq2_aq2_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq2/re

In [15]:
model_name = train(w_nbits=2, a_nbits=3)


Quantization: weight=2 activation=3, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [00:53<00:00,  7.33batch/s]


LR: 1.0000e-01, Train Loss: 1.7149, Train Acc: 0.3607, Train Latency: 53.35s, Memory Usage: 332.31MB
Test Loss: 1.4998, Test Acc: 0.4265, Test Latency: 7.32s, Test Memory Usage: 124.55MB

Epoch 2


Train: 100%|##########| 391/391 [00:54<00:00,  7.20batch/s]


LR: 1.0000e-01, Train Loss: 1.3282, Train Acc: 0.5168, Train Latency: 54.31s, Memory Usage: 332.81MB
Test Loss: 1.5928, Test Acc: 0.4666, Test Latency: 7.38s, Test Memory Usage: 126.55MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.2415, Train Acc: 0.9176, Train Latency: 54.56s, Memory Usage: 329.79MB
Test Loss: 0.3406, Test Acc: 0.8848, Test Latency: 7.34s, Test Memory Usage: 132.93MB

Epoch 199


Train: 100%|##########| 391/391 [00:54<00:00,  7.19batch/s]


LR: 1.0000e-04, Train Loss: 0.2402, Train Acc: 0.9183, Train Latency: 54.37s, Memory Usage: 329.78MB
Test Loss: 0.3474, Test Acc: 0.8842, Test Latency: 7.50s, Test Memory Usage: 132.96MB

Epoch 200


Train: 100%|##########| 391/391 [00:55<00:00,  7.00batch/s]


LR: 1.0000e-04, Train Loss: 0.2413, Train Acc: 0.9189, Train Latency: 55.90s, Memory Usage: 329.79MB
Test Loss: 0.3400, Test Acc: 0.8825, Test Latency: 7.58s, Test Memory Usage: 132.82MB
Saving final epoch model ...


In [16]:
analyze_model(model_name)

weight bits: 2 activation bits: 3 base filename: resnet20_cifar_wq2_aq3
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/resnet20_cifar_wq2_aq3_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq2_aq3/re

In [17]:
model_name = train(w_nbits=3, a_nbits=3)


Quantization: weight=3 activation=3, Using device: cuda

Epoch 1


Train: 100%|##########| 391/391 [01:01<00:00,  6.31batch/s]


LR: 1.0000e-01, Train Loss: 1.6983, Train Acc: 0.3700, Train Latency: 61.99s, Memory Usage: 332.26MB
Test Loss: 1.7476, Test Acc: 0.4022, Test Latency: 8.86s, Test Memory Usage: 126.55MB

Epoch 2


Train: 100%|##########| 391/391 [01:02<00:00,  6.28batch/s]


LR: 1.0000e-01, Train Loss: 1.2669, Train Acc: 0.5419, Train Latency: 62.30s, Memory Usage: 332.76MB
Test Loss: 1.4369, Test Acc: 0.4811, Test Latency: 8.81s, Test Memory Usage: 126.82MB

Epoch 3


...

LR: 1.0000e-04, Train Loss: 0.1990, Train Acc: 0.9332, Train Latency: 59.73s, Memory Usage: 329.81MB
Test Loss: 0.3250, Test Acc: 0.8909, Test Latency: 8.49s, Test Memory Usage: 129.03MB

Epoch 199


Train: 100%|##########| 391/391 [01:00<00:00,  6.50batch/s]


LR: 1.0000e-04, Train Loss: 0.2022, Train Acc: 0.9320, Train Latency: 60.12s, Memory Usage: 329.82MB
Test Loss: 0.3350, Test Acc: 0.8910, Test Latency: 8.66s, Test Memory Usage: 129.32MB

Epoch 200


Train: 100%|##########| 391/391 [01:00<00:00,  6.49batch/s]


LR: 1.0000e-04, Train Loss: 0.2015, Train Acc: 0.9315, Train Latency: 60.21s, Memory Usage: 329.81MB
Test Loss: 0.3370, Test Acc: 0.8862, Test Latency: 8.51s, Test Memory Usage: 129.08MB
Saving final epoch model ...


In [18]:
analyze_model(model_name)

weight bits: 3 activation bits: 3 base filename: resnet20_cifar_wq3_aq3
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.0.conv1_and_layer1.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.0.conv2_and_layer1.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.1.conv1_and_layer1.1.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.1.conv2_and_layer1.1.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.2.conv1_and_layer1.2.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer1.2.conv2_and_layer1.2.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer2.0.conv1_and_layer2.0.activ1.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/resnet20_cifar_wq3_aq3_layer2.0.conv2_and_layer2.0.activ2.1.png
Saved plot as ./output/resnet20_cifar_wq3_aq3/re